# Machine Learning In Production

This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# Let's make sure we are running the latest version of the SakeMaker's SDK. 
# Restart the notebook after you upgrade the library.

!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker
!pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.22 requires botocore==1.23.22, but you have botocore 1.29.111 which is incompatible.
awscli 1.22.22 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.6.0 which is incompatible.
Name: sagemaker
Version: 2.145.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, protobuf3-to-dict, PyYAML, schema, smdebug-rulesconfig
Required-by: 


# Session 1 - Getting Started

This session aims to build a simple [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with one step to preprocess the dataset.


In [5]:
import os
import sagemaker
import numpy as np
import boto3
import json
import pandas as pd
import numpy as np
import urllib.request
import argparse
import tempfile
from pathlib import Path

from botocore.exceptions import ClientError
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig


role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Step 1 - Creating an S3 Bucket

We need to create an S3 bucket where we will upload everything we need during the program.

Make sure you set `BUCKET` to the name of the bucket you want to use.

In [3]:
BUCKET = "mlschool"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool"
}


## Step 2 - Downloading the Dataset

We can now download the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and store it in S3.

In [4]:
S3_FILEPATH = f"s3://{BUCKET}/penguins"
DATA_FILEPATH = "penguins/data.csv"

# Download the official Penguins dataset and store it locally.
urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

# Upload the dataset to S3. We need to do this to make it available to 
# the preprocessing step.
INPUT_DATA_URI = sagemaker.s3.S3Uploader.upload(
    local_path=DATA_FILEPATH, 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Dataset S3 location: {INPUT_DATA_URI}")

Dataset S3 location: s3://mlschool/penguins/data.csv


We can now load and display the dataset.

In [5]:
df = pd.read_csv(DATA_FILEPATH)
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


## Step 3 - Preprocessing the Dataset

Let's create a script to do feature engineering on the original dataset. This script should also split the data into train, validation, and a test set.

In [6]:
%%writefile penguins/preprocessor.py

import os
import numpy as np
import pandas as pd
import tempfile

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler


BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIR) / "input" / "data.csv"


def save_splits(base_dir, train, validation, test):
    """
    Saves the supplied datasets to disk.
    """
    
    train_path = Path(base_dir) / "train" 
    validation_path = Path(base_dir) / "validation" 
    test_path = Path(base_dir) / "test"
    
    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)


def preprocess(base_dir, data_filepath):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """
    
    df = pd.read_csv(data_filepath)
    
    numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]

    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
            ("categorical", categorical_preprocessor, ["island"]),
        ]
    )
    

    X = df.drop(["sex"], axis=1)
    columns = list(X.columns)
    
    X = X.to_numpy()
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])
    
    X_train = pd.DataFrame(train, columns=columns)
    X_validation = pd.DataFrame(validation, columns=columns)
    X_test = pd.DataFrame(test, columns=columns)
    
    y_train = X_train.species
    y_validation = X_validation.species
    y_test = X_test.species

    X_train.drop(["species"], axis=1, inplace=True)
    X_validation.drop(["species"], axis=1, inplace=True)
    X_test.drop(["species"], axis=1, inplace=True)
    
    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    label_encoder = LabelEncoder()

    y_train = label_encoder.fit_transform(y_train)
    y_validation = label_encoder.transform(y_validation)
    y_test = label_encoder.transform(y_test)
    
    
    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)
    
    save_splits(base_dir, train, validation, test)
    
    
if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH)


Overwriting penguins/preprocessor.py


## Step 4 - Testing the Preprocessing Script

We can now load the script we just created and run it locally to ensure it creates the 3 splits. 

Having a way to run scripts locally is crucial to shorten the development feedback.

In [9]:
from penguins.preprocessor import preprocess

with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    print(f"Splits: {os.listdir(directory)}")
    print(f"Train: {os.listdir(Path(directory) / 'train')}")
    print(f"Validation: {os.listdir(Path(directory) / 'validation')}")
    print(f"Test: {os.listdir(Path(directory) / 'test')}")

Splits: ['train', 'validation', 'test']
Train: ['train.csv']
Validation: ['validation.csv']
Test: ['test.csv']


## Step 5 - Pipeline Configuration

When we create a SageMaker Pipeline we can specify a list of paramaters that we can use throughout the individual pipeline steps. To read more about these parameters, check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html).

These are the parameters that will use in our pipeline:

* `dataset_location`: This parameter represents the location of the dataset in S3. We will use this parameter during the preprocessing step to access the dataset.
* `preprocessor_destination`: We need to define the location where the preprocessing step will be storing the dataset splits to avoid SageMaker from appending a timestamp to their auto-generated location. If we let SageMaker use a timestamp, we can't cache this step.

In [10]:
dataset_location = ParameterString(
    name="dataset_location",
    default_value=INPUT_DATA_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f'{S3_FILEPATH}/preprocessing',
)

## Step 6 - Caching Pipeline Steps

While you are building your pipeline, you don't want to rerun every step of the process unless you expect a different result. Instead, you can instruct SageMaker to reuse the result of a previous successful run of a pipeline step.

You can accomplish this by caching your steps. You can find more information about this topic in [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html).

Getting caching to work is tricky, and you will find SageMaker missing the cache frequently. Whenever that happens, you need to dig and figure out how to adjust the step configuration to prevent SageMaker from autogenerating data that prevents a cache hit. For example, to cache the preprocessing step we need to define the destination of the processing job to prevent SageMaker from using an autogenerated timestamp.

In [11]:
# We'll use this cache configuration to cache individual steps for 
# a maximum of 15 days.
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

## Step 7 - Setting up a Processing Step

The first step we need in our pipeline is a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) to run the preprocessing script. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

To run our script, we need access to Scikit-Learn, so we can use the [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) processor that comes out-of-the-box with the SageMaker's Python SDK.

The input of this step will be the dataset location, and the output will be the location of the three sets.

In [12]:
sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

preprocess_step = ProcessingStep(
    name="penguins-preprocessing-step",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
    ],
    code="penguins/preprocessor.py",
    cache_config=cache_config
)

## Step 8 - Defining and Running the Pipeline

We can now define and run the SageMaker Pipeline. Check [Pipeline Structure and Execution](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-pipeline.html) for more information about how to define a pipeline and [Run a Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/run-pipeline.html) for information about how to run it.


In [13]:
session1_pipeline = Pipeline(
    name="session1-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [14]:
session1_pipeline.upsert(role_arn=role)
execution = session1_pipeline.start()

## Step 9 - Cleaning up

Before you finish, don't forget to clean up after you.

In [16]:
session1_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session1-penguins-pipeline',
 'ResponseMetadata': {'RequestId': 'f5e5d16b-0101-4cc7-b44c-5dfc2736a658',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f5e5d16b-0101-4cc7-b44c-5dfc2736a658',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Tue, 11 Apr 2023 20:01:15 GMT'},
  'RetryAttempts': 0}}

## Assignments

1. Set up an Amazon SageMaker domain using the Standard Setup. Make sure you set the network configuration to VPC Only. Create a new execution role and ensure it has access to the S3 bucket you’ll use during this class. You can also specify “Any S3 bucket” if you want this role to access every S3 bucket in your AWS account.

2. Create a GitHub repository and clone it from inside SageMaker Studio. We’ll use this repository to store the code used during this program.

3. Configure your SageMaker Studio session to store your name and email address and cache your credentials. You can use the following commands from a Terminal window:

```bash
$ git config --global user.name "John Doe"
$ git config --global user.email johndoe@example.com
$ git config --global credential.helper store
```

4. Prepare the MNIST dataset. Throughout the course, you will create a SageMaker pipeline to work with the MNIST dataset. MNIST is popular and relatively small, so it's easy to find pre-packaged versions of it. We aren't going to use those. Instead, we will simulate a practical scenario where the data is stored in the filesystem. To accomplish this, you will load a pre-packaged version of MNIST, save it to disk, and upload it to an S3 bucket. Complete the section "Prepare the MNIST Dataset" below.

5. Setup a SageMaker Pipeline with a preprocessing step where you split the MNIST dataset into a train and a test set.

### Prepare the MNIST Dataset

These are the steps you need to follow to prepare the data:

1. Create the S3 bucket to upload the dataset.
2. Load the MNIST dataset from the Keras built-in collection of small datasets, convert it into images, and save them to the disk.
3. Upload the dataset to the S3 bucket.


#### Create the dataset

We want to load the Keras' built-in version of MNIST and save it to disk so we can later upload it to S3.

There are 70,000 images. Running this cell will take some time, so this is the perfect moment to walk around and grab some coffee. Fortunately, we only need to do this once.

In [ ]:
import numpy as np
import tensorflow as tf

from PIL import Image
from pathlib import Path
from tensorflow.keras.datasets import mnist


def save(dataset, split, images, labels):
    """
    This function saves the handwritten digits to disk as PNG files.
    
    Every image will be saved inside a folder corresponding to 
    its label. For example, a digit from the train set representing 
    the number 3 will be saved inside the folder `~/train/3`.
    """
    
    for index, (image, label) in enumerate(zip(images, labels)):
        im = Image.fromarray(image)

        path = dataset / split / str(label)
        path.mkdir(parents=True, exist_ok=True)
        
        im.save(path / f"{index}.png")
        

# We will save the dataset in the home directory, inside a folder
# named `dataset`.
dataset = Path.home() / "dataset" 

# We want to make sure we don't generate the images if the dataset
# already exists.
if not dataset.exists():
    # Load the MNIST dataset using the Keras library. This returns the
    # dataset in numpy arrays.
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # Use the function we created to save the data to disk.
    save(dataset, split="train", images=X_train, labels=y_train)
    save(dataset, split="test", images=X_test, labels=y_test)

#### Upload the data to S3

Now that we exported the MNIST dataset to the filesystem, we need to upload them to an S3 bucket. The easiest way to do this is to use the AWS CLI.

This command will also take a while to finish.

In [ ]:
!aws s3 cp $dataset s3://$BUCKET/mnist --recursive

## References

1. Amazon SageMaker is free to try. Your free tier starts from the first month when you create your first SageMaker resource and lasts 2 months. Check out the [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) for more information.

2. We’ll be working extensively with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) and [SageMaker’s Python SDK](https://sagemaker.readthedocs.io/en/stable/). Keep their documentation handy.

3. Check the [SageMaker Pipelines Overview](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) for an introduction to the fundamental components of a SageMaker Pipeline.

4. Check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) for more information on how to define and use variables in your pipeline.

5. Check [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html) for information on how to cache the results of individual pipeline steps.

6. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation. You can find an example of how to create a processing job from the pipeline in the [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) page.

7. Check [Pipeline Structure and Execution](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-pipeline.html) for more information about how to define a pipeline.

8. Check [Run a Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/run-pipeline.html) for information about how to submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does, and then run it.

## Additional Notes

1. This notebook uses a Scikit-Learn Pipeline to transform the dataset. You should always orchestrate your transformations using pipelines. Check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for more details.

2. The preprocessing script uses `np.split()` to split the dataset into 3 different splits. It's a neat way of getting the three splits with a single instruction.

3. Keras offers a [list of built-in vectorized datasets](https://www.notion.so/Bnomial-RESTful-API-4ecf85043b484ec994d7f70c56abfe27) in NumPy format. You can load any of these datasets with a single line of code, making them convenient.

4. Converting a Numpy array into an image you can save and visualize is a useful trick to know. Check the `Image.fromarray()` function from the `PIL` library.

5. The [command line interface](https://docs.aws.amazon.com/cli/latest/index.html) is a simple way to interact with the AWS services. You can combine Python code with bash commands in the same notebook cell, which makes notebooks a very flexible tool.

6. Check Python’s `pathlib` module. Since Python 3.4, this module offers a clean way to interact with the filesystem.

7. This notebook uses the `%%writefile`, `%load_ext`, and `%autoreload` magics. These magics are very useful when using notebooks. Check this list of [line and cell magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) for other examples.

# Session 2 - Training and Tuning

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) we built in the previous session with one step that trains a model.

We explore the [Training](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) and the [Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) steps.


In [17]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession

## Step 1 - Training the Model

This script is responsible from training a simple neural network on the train data, validating the model, and saving it so we can later use it.

In [18]:
%%writefile penguins/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list: 
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting penguins/train.py


## Step 2 - Testing the Training Script

Let's test the script we just created by running it locally.

In [19]:
from penguins.preprocessor import preprocess
from penguins.train import train


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )

Epoch 1/50
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2023-04-11 20:01:49.420 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:236 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-04-11 20:01:49.510 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:236 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
8/8 - 1s - loss: 1.0310 - accuracy: 0.5314 - val_loss: 0.9874 - val_accuracy: 0.6078
Epoch 2/50
8/8 - 0s - loss: 0.9778 - accuracy: 0.6067 - val_loss: 0.9337 - val_accuracy: 0.7451
Epoch 3/50
8/8 - 0s - loss: 0.9282 - accuracy: 0.6653 - val_loss: 0.8834 - val_accuracy: 0.7647
Epoch 4/50
8/8 - 0s 

INFO:tensorflow:Assets written to: /tmp/tmp2knmct_p/model/001/assets


## Step 3 - Configuring an Estimator

SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the trainning script we wrote before.

SageMaker will pass the list of hyperparameters defined below to the entry point of the training script as arguments.

In [20]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32
}

estimator = TensorFlow(
    entry_point="penguins/train.py",
    role=role,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.large",
    instance_count=1,
    py_version="py37",
    framework_version="2.4",
    script_mode=True,
    disable_profiler=True
)

## Step 4 - Setting up a Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information. 

This step will use the estimator we configured before and will receive the train and validation splits from the preprocessing step as inputs.

In [21]:
training_step = TrainingStep(
    name="penguins-training-step",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 5 - Running the Pipeline with the Training Step

We can now define and run the SageMaker Pipeline, this time using the new Training Step.

In [22]:
session2_training_pipeline = Pipeline(
    name="session2-training-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        training_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [23]:
session2_training_pipeline.upsert(role_arn=role)
execution = session2_training_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Step 6 - Configuring a Hyperparameter Tuner

An alternative to training a model is to train many variants of the model and choose the best one. We can do this with an instance of the [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) class.

The tuner uses the same `Estimator` we defined to train the model, and we can specify how it should determine the best model:

1. `objective_metric_name`: This is the name of the metric the tuner will use to determine the best model.
2. `objective_type`: This is the objective of the tuner. Should it "Minimize" the metric or "Maximize" it? In this example, sice we are using the validation accuracy of the model, we want the objetive to be "Maximize." If we were using the loss of the model, we would set the objetive to "Minimize."
3. `metric_definitions`: Defines how the tuner will determine the value of the metric by looking at the output logs of the training process.

The tuner expects a list of the hyperparameters you want to explore. You can use subclasses of the [Parameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange) class to specify different types of hyperparameters.

Finally, you can control the number of jobs and how many of them will run in parallel using the following two arguments:
* `max_jobs`: Defines the maximum total number of training jobs to start for the hyperparameter tuning job.
* `max_parallel_jobs`: Defines the maximum number of parallel training jobs to start.

In [24]:
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 50)
}

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

## Step 7 - Setting up a Tuning Step

We can now create a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) to add it to our pipeline. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation for more information. 

This step will use the tuner we configured before and will receive the train and validation splits from the preprocessing step as inputs.

In [25]:
tuning_step = TuningStep(
    name = "penguins-tuning-step",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 8 - Running the Pipeline with the Tuning Step

We can now define and run the SageMaker Pipeline, this time using the new Tuning Step.

In [26]:
session2_tuning_pipeline = Pipeline(
    name="session2-tuning-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [27]:
session2_tuning_pipeline.upsert(role_arn=role)
execution = session2_tuning_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Step 9 - Cleaning up

Before you finish, don't forget to clean up after you.

In [29]:
session2_training_pipeline.delete()
session2_tuning_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session2-tuning-penguins-pipeline',
 'ResponseMetadata': {'RequestId': 'fb4b7e41-1fb4-4077-887b-15ca1bf09910',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb4b7e41-1fb4-4077-887b-15ca1bf09910',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Tue, 11 Apr 2023 20:35:46 GMT'},
  'RetryAttempts': 2}}

## Assignments

1. Modify the training script so it accepts the `learning_rate` as a new hyperparameter using the list of hyperparameters supplied to the Estimator.

2. Replace the TensorFlow Estimator with a Pytorch Estimator. Check [this page](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#create-an-estimator) for an example of how to create a PyTorch Estimator. You'll need to create a new training script that builds a PyTorch model to solve the problem.

3. Modify the tuner job to find the best `learning_rate` value between `0.01` and `0.03`. Check the [ContinuousParameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ContinuousParameter) class for more information on how to configure this parameter.

4. Modify the pipeline to run the training and tuning jobs concurrently.

5. Modify the SageMaker Pipeline you created for the MNIST project and add a training step. That step should only receive one channel with the train data. 

## References

1. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available fremwork versions for each region.

2. Check [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit) for more information about how to train machine learning models within a Docker container using Amazon SageMaker.

3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation. You can find an example of how to create a training job from the pipeline in the [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) page.

4. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation. You can find an example of how to create a hyperparameter tuning job from the pipeline in the [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) page.

5. Check the [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) and the [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) documentation for more information about how these classes work. You can also find [other supported frameworks](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html) in the documentation.

6. Check the [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) class for more information about how to configure a hyperparameter job.

7. The SageMaker SDK passes special hyperparameters to the training job that we can capture from inside the script. Here is the [complete list of available hyperparameters](https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py). 



# Session 3 - Evaluating the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to evaluate the model.


In [14]:
import tarfile

from sagemaker.workflow.properties import PropertyFile

## Step 1 - Evaluating the Model

This script is reponsible from loading the model we created and evaluating it on the test set. Before finishing, this script will create a file containing an evaluation report of the model.

In [7]:
%%writefile penguins/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package provided
    # by SageMaker.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    # We can now load the model from disk.
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's add the accuracy of the model to our evaluation report.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    # We need to save the evaluation report to the output path.
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting penguins/evaluation.py


## Step 2 - Testing the Evaluation Script

Let's test the script we just created by running it locally.

In [ ]:
from penguins.preprocessor import preprocess
from penguins.train import train
from penguins.evaluation import evaluate


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=50
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
    )

## Step 3 - Setting up a Processor

To run the evaluation script we can use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing). Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

This time, we will use a [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) running a TensorFlow image. This will give us access to every library we need to execute the evaluation script.

You can use the [sagemaker.image_utis.retrieve()](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html) function for generating the URI of pre-built docker images.

In [31]:
# Let's retrieve the image we want to use to run the
# processing job.
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    py_version="py37",
    image_scope="training",
    instance_type="ml.m5.large"
)

# We can now setup the processor using the URI of
# the pre-built docker image.
evaluation_script_processor = ScriptProcessor(
    base_job_name="penguins-evaluation-processor",
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

## Step 4 - Configuring the Model Input

One of the inputs to the Evaluation Step is the model we created. We explored two different ways to create the model: a Training Step and a Tuning Step.

Here we can configure the input to the Evaluation Step based on whether we want to select the best model generated by the Tuning Step, or the model we trained using the Training Step.

We can use the [get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model artifacts from the top performing training jobs of the hyperparameter tuning job.

In [35]:
# By default, this notebook uses the best model from the Tuning Step.
# You can set this variable to False if you want to use the result
# of the Training Step.
USE_TUNING_STEP = True

# This is the input in case we want to use the best model generated
# by the Tuning Step.
tuning_model_input = ProcessingInput(
    source=tuning_step.get_top_model_s3_uri(
        top_k=0, 
        s3_bucket=sagemaker_session.default_bucket()
    ),
    destination="/opt/ml/processing/model",
)

# This is the input in case we want to use the trained model
# from the Training Step.
training_model_input = ProcessingInput(
    source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    destination="/opt/ml/processing/model"
)

# We can now select the appropriate input depending on which step
# we are using.
model_input = tuning_model_input if USE_TUNING_STEP else training_model_input

## Step 5 - Setting up a Processing Step

We can now create a [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) to run the evaluation script. We'll use the [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) we defined before. 

The inputs of this step will be the model and the test set that we generated during the preprocessing step. The output will be the evaluation report file.

The [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) lets us specify a list of [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile) instances from the output of the job. We can use this to map the evaluation report that we generate in the evaluations script. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.

In [ ]:
# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluation_step = ProcessingStep(
    name="penguins-evaluation-step",
    processor=evaluation_script_processor,
    inputs=[
        model_input,
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="penguins/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config
)

## Step 6 - Adding Model Evaluation to the Pipeline

We can now add the model evaluation step to the pipeline.

We are going to configure the pipeline to run the Tuning Step or the Training Step depending on the value of the `USE_TUNING_STEP` flag.

In [39]:
session3_pipeline = Pipeline(
    name="session3-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step,
        evaluation_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [40]:
session3_pipeline.upsert(role_arn=role)
execution = session3_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Step 7 - Cleaning up

Before you finish, don't forget to clean up after you.

In [ ]:
session3_pipeline.delete()

## Assignments

1. Extend the evaluation report by adding other metrics. For example, add the support of the test set (the number of samples.)

2. One of the assignments from the previous session was to replace the TensorFlow Estimator with a Pytorch Estimator. You can now modify the evaluation step to load a script that uses Pytorch to evaluate the model.

3. If you are runing the Training and Tuning Steps simultaneously, create two different Evaluation Steps to evaluate both models independently.

4. Instead of runing the Training and Tuning Steps simultaneously, run the Tuning Step but create two Evaluation Steps to evaluate the two best models produced by the Tuning Step. Check the [TuningJob.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to retrieve the two best models.

5. Modify the SageMaker Pipeline you created for the MNIST project and add an evaluation step. That step should use the test set you generated in the preprocessing step.

## Resources

1. Check the [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) for more information about how to run a processing job using a machine learning framework.

2. SageMaker offers a list of pre-built docker images. You can use the [sagemaker.image_utis.retrieve()](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html) function for generating the URI of these images.

3. You can use the [TuningJob.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model artifacts from the top performing training jobs of the hyperparameter tuning job.

4. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information about mapping the output of a [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) to a [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile).

# Session 4 - Deploying the Model

In [78]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker import ModelPackage
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join

## Step 1 - Preparing Pipeline Parameters

In [42]:

# This represents the approval status of a new trained model in the registry.
model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

# This property represents the minimum accuracy that the model should
# reach in order for it to be registered.
accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.75
)


## Step 2 - Configuring the Model Assets

In [43]:
training_model_data = training_step.properties.ModelArtifacts.S3ModelArtifacts

tuning_model_data = tuning_step.get_top_model_s3_uri(
    top_k=0, 
    s3_bucket=sagemaker_session.default_bucket()
)

model_data = tuning_model_data if USE_TUNING_STEP else training_model_data

## Step 3 - Configuring the Model

In [48]:
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    image_scope="inference",
    instance_type="ml.m5.large"
)

model = Model(
    image_uri=image_uri,
    model_data=model_data,
    sagemaker_session=PipelineSession(),
    role=role,
)

## Step 4 - Setting up a Model Step

In [76]:
model_package_group_name = "penguins-model-package-group"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "/evaluation.json"]
        ),
        content_type="application/json",
    )
)

model_step = ModelStep(
    name="penguins-model-step",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    ),
)

## Step 5 - Setting up a Conditional Step

In [79]:
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

register_model_step = ConditionStep(
    name="penguins-register-model-step",
    conditions=[condition_gte],
    if_steps=[model_step],
    else_steps=[], 
)

## Step 6 - Adding Model Registration to the Pipeline

In [80]:
session4_pipeline = Pipeline(
    name="session4-penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step, 
        evaluation_step,
        register_model_step
    ],
)

In [81]:
session4_pipeline.upsert(role_arn=role)
execution = session4_pipeline.start()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


## Step 7 - Loading the Latest Approved Model

In [ ]:
sagemaker_client = boto3.client("sagemaker")


def get_latest_approved_model_package(model_package_group_name):
    """
    Returns the latest approved model package registered under the 
    supplied model package group.
    """
    try:
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        while len(approved_packages) == 0 and "NextToken" in response:
            response = sagemaker_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        if len(approved_packages) == 0:
            error = f"No approved model pacakages for {model_package_group_name}"
            print(error)
            raise Exception(error)

        # Return the pmodel package arn
        model_package_arn = approved_packages[0]["ModelPackageArn"]
        logger.info(f"Identified the latest approved model package: {model_package_arn}")
        return approved_packages[0]

    except ClientError as e:
        print(e.response["Error"]["Message"])
        raise Exception(e.response["Error"]["Message"])
        
        
def does_endpoint_exist(endpoint_name):
    """
    Returns whether the supplied endpoint already exists.
    """
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        return True
    except ClientError as e:
        return False

In [ ]:
approved_model_package = get_latest_approved_model_package(model_package_group_name)
approved_model_package_arn = approved_model_package["ModelPackageArn"]

model_description = sagemaker_client.describe_model_package(
    ModelPackageName=approved_model_package_arn
)

model_description

## Step 8 - Deploying the Model

In [ ]:
model_package = ModelPackage(
    role=role, 
    model_package_arn=approved_model_package_arn, 
    sagemaker_session=sagemaker_session
)

endpoint_name = "penguins-endpoint"

if not does_endpoint_exist(endpoint_name):
    model_package.deploy(
        initial_instance_count=1, 
        instance_type="ml.m5.large", 
        endpoint_name=endpoint_name
    )

## Step 9 - Testing the Model Endpoint

In [ ]:
predictor = Predictor(endpoint_name=endpoint_name)

payload = "0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0"
p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})

predictions = json.loads(p.decode("utf-8"))["predictions"]

print(f"Prediction: {np.argmax(predictions)}")

## Step 11 - Cleaning up

In [82]:
for model_package in sagemaker_client.list_model_packages(
    ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(f"Deleting {model_package['ModelPackageArn']}")
    sagemaker_client.delete_model_package(ModelPackageName=model_package["ModelPackageArn"])

    
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)
predictor.delete_endpoint()
session4_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session4-penguins-pipeline',
 'ResponseMetadata': {'RequestId': '0dccb461-d532-4e0e-a21c-957e31558a4f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0dccb461-d532-4e0e-a21c-957e31558a4f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Tue, 11 Apr 2023 22:12:41 GMT'},
  'RetryAttempts': 0}}

## Assignments

## References

# Session 5 - Custom Endpoints

In [14]:
import uuid
from datetime import datetime

In [15]:
VERSION = "1"

ENDPOINTS_FOLDER = "version" + VERSION
REPOSITORY_NAME = "penguins"

CONTAINER_VERSION = f"{datetime.now().year}{str(datetime.now().month).zfill(2)}{str(datetime.now().day).zfill(2)}{str(uuid.uuid4())[-4:]}" 
print(f"Container version: {CONTAINER_VERSION}")

Container version: 20230412a9c6


In [16]:
!aws ecr create-repository --repository-name $REPOSITORY_NAME


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'penguins' already exists in the registry with id '325223348818'


In [18]:
repository = !aws ecr describe-repositories \
    --repository-names $REPOSITORY_NAME \
    --query "repositories[0].repositoryUri"

repository_uri = repository[0][1:-1]
repository = repository_uri[0:repository_uri.index("/")]
print(f"ECR Repository: {repository}/{REPOSITORY_NAME}")

ECR Repository: 325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins


In [20]:
AWS_ACCOUNT_ID = !aws sts get-caller-identity --query "Account" --output text
AWS_ACCOUNT_ID = AWS_ACCOUNT_ID[0] 


print(f"cd ~/ml.school/container")
print(f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {AWS_ACCOUNT_ID}.dkr.ecr.{region}.amazonaws.com")
print(f"docker build -t {repository_uri}:{CONTAINER_VERSION} .")

cd ~/ml.school/container
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 325223348818.dkr.ecr.us-east-1.amazonaws.com
docker build -t 325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins:20230412a9c6 .


In [79]:
ecr = f"{AWS_ACCOUNT_ID}.dkr.ecr.{region}.amazonaws.com"

!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr


# docker build --build-arg MULTI_MODEL=false -t $repository_uri:$CONTAINER_VERSION .
    
# aws ecr get-login-password | docker login --username AWS --password-stdin $repository
# docker push $repository_uri:$CONTAINER_VERSION

/usr/bin/sh: 1: docker: not found


In [3]:
!sdocker

/usr/bin/sh: 1: sdocker: not found


In [63]:
region

'us-east-1'

# Notes

* [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit)
* [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html)